In [1]:
import sympy

def rat(expr):
    for i in expr.atoms(sympy.Float):
        r = sympy.Rational(str(i)).limit_denominator(1000)
        expr = expr.subs(i, r)
    return expr


In [2]:
from IPython.display import display, Markdown
import sympy
from sympy import *
import numpy as np
x, xi, u = symbols(r'x \xi u', real=True)
d1, d2, d3 = symbols(r'd_1 d_2 d_3', real=True)

Xi = [ -1/sqrt(3), 1/sqrt(3) ]
W = [ 1, 1 ]

kappa = Function(r"K")(xi)
dkappa_du = Function(r"K_{,u}")(xi)

Na = [
1/2*xi*(xi-1),
1-xi*xi,
1/2*xi*(xi+1)
]

display( Markdown( f"$N_1 = {sympy.latex(Na[0])}$" ))
display( Markdown( f"$N_2 = {sympy.latex(Na[1])}$" ))
display( Markdown( f"$N_3 = {sympy.latex(Na[2])}$" ))

$N_1 = 0.5 \xi \left(\xi - 1\right)$

$N_2 = 1 - \xi^{2}$

$N_3 = 0.5 \xi \left(\xi + 1\right)$

In [3]:
dN1_xi = diff(Na[0], xi)
dN2_xi = diff(Na[1], xi)
dN3_xi = diff(Na[2], xi)

display( Markdown( r"$dN_1/d\xi = "+ f"{sympy.latex(dN1_xi)}$" ))
display( Markdown( r"$dN_2/d\xi = "+ f"{sympy.latex(dN2_xi)}$" ))
display( Markdown( r"$dN_3/d\xi = "+ f"{sympy.latex(dN3_xi)}$" ))


$dN_1/d\xi = 1.0 \xi - 0.5$

$dN_2/d\xi = - 2 \xi$

$dN_3/d\xi = 1.0 \xi + 0.5$

In [4]:
X = Na[1] * 0.5 + Na[2]
X = simplify(X)
dx_dxi = diff(X,xi)
dxi_dx = 1/dx_dxi

display( Markdown( r"$x(\xi) = "+ f"{sympy.latex(X)}$"))
display( Markdown( r"$dx/d\xi = "+ f"{sympy.latex(dx_dxi)}$" ))
display( Markdown( r"$d\xi/dx = "+ f"{sympy.latex(dxi_dx)}$" ))


$x(\xi) = 0.5 \xi + 0.5$

$dx/d\xi = 0.5$

$d\xi/dx = 2.0$

In [5]:
dN1_x = dN1_xi * dxi_dx 
dN2_x = dN2_xi * dxi_dx 
dN3_x = dN3_xi * dxi_dx 

display( Markdown( r"$dN_1/dx = "+ f"{sympy.latex(dN1_x)}$" ))
display( Markdown( r"$dN_2/dx = "+ f"{sympy.latex(dN2_x)}$" ))
display( Markdown( r"$dN_3/dx = "+ f"{sympy.latex(dN3_x)}$" ))


$dN_1/dx = 2.0 \xi - 1.0$

$dN_2/dx = - 4.0 \xi$

$dN_3/dx = 2.0 \xi + 1.0$

In [6]:
def build_N_x( xi_ ) :
    # Derivatives in X space
    N1_x = dN1_xi.subs(xi, xi_) * dxi_dx
    N2_x = dN2_xi.subs(xi, xi_) * dxi_dx
    N3_x = dN3_xi.subs(xi, xi_) * dxi_dx
    return N1_x, N2_x, N3_x

db = [ d1, d2, d3 ]
ne = [ 0, 0, 0 ]

f = symbols(r'f_1 f_2 f_3', real=True)
h = symbols(r'h', real=True)
fe = [ 0, 0, 0 ]
dna_ddb = zeros( 3, 3)
fcol = []
for xi_, W_ in zip( Xi, W ) :
    Na_x_ = build_N_x(xi_)
    x_ = X.subs(xi, xi_)
    kappa_ = kappa.subs( xi, xi_ )
    dkappa_du_ = dkappa_du.subs( xi, xi_ )
    fcol.append(kappa_)
    fcol.append(dkappa_du_)
    Na_ = [0, 0, 0]
    for a in range(3) : Na_[a] = simplify(Na[a].subs(xi, xi_))

    q = 0
    for b in range(3) : q += db[b] * Na_x_[b]
    for a in range(3) :
        ne[a] += W_ * dx_dxi * Na_x_[a] * q * kappa_
        fe[a] += W_ * dx_dxi * Na_[a] * f[a]
        fe[a] = simplify(fe[a])
    
    for a in range(3) :
        for b in range(3) :
            dna_ddb[a,b] += W_ * dx_dxi * Na_x_[a] * Na_[b] * q * dkappa_du_
            dna_ddb[a,b] += W_ * dx_dxi * kappa_ * Na_x_[a] * Na_x_[b]
            
    fe[0] += W_ * h * Na_[0]
    fe[0] = simplify(fe[0])
    
for a in range(3) :
    ne[a] = cancel(ne[a])
    for c in fcol : ne[a] = collect(ne[a],c)
    for d in db : ne[a] = collect( ne[a], d )
    ne[a] = simplify(ne[a],full=True)

for a in range(3) :
    fe[a] = nsimplify(fe[a])

for a in range(3) :
    for b in range(3) :
        dna_ddb[a,b] = cancel(dna_ddb[a,b])
        for c in fcol : dna_ddb[a,b] = collect(dna_ddb[a,b],c)
        for d in db : dna_ddb[a,b] = collect( dna_ddb[a,b], d )
        dna_ddb[a,b] = simplify(dna_ddb[a,b], full=True)


In [7]:
for a in range(3) :
    display( Markdown( f"$n_{a+1}^e(d^e) = {sympy.latex(rat(ne[a]))}$" ))

$n_1^e(d^e) = \left(d_{1} \left(\frac{7}{6} - \frac{2 \sqrt{3}}{3}\right) - d_{2} \left(\frac{4}{3} - \frac{2 \sqrt{3}}{3}\right) + \frac{d_{3}}{6}\right) K{\left(\frac{\sqrt{3}}{3} \right)} + \left(d_{1} \left(\frac{2 \sqrt{3}}{3} + \frac{7}{6}\right) - d_{2} \left(\frac{2 \sqrt{3}}{3} + \frac{4}{3}\right) + \frac{d_{3}}{6}\right) K{\left(- \frac{\sqrt{3}}{3} \right)}$

$n_2^e(d^e) = - \left(d_{1} \left(\frac{4}{3} - \frac{2 \sqrt{3}}{3}\right) - \frac{8 d_{2}}{3} + d_{3} \left(\frac{2 \sqrt{3}}{3} + \frac{4}{3}\right)\right) K{\left(\frac{\sqrt{3}}{3} \right)} - \left(d_{1} \left(\frac{2 \sqrt{3}}{3} + \frac{4}{3}\right) - \frac{8 d_{2}}{3} + d_{3} \left(\frac{4}{3} - \frac{2 \sqrt{3}}{3}\right)\right) K{\left(- \frac{\sqrt{3}}{3} \right)}$

$n_3^e(d^e) = \left(\frac{d_{1}}{6} - d_{2} \left(\frac{4}{3} - \frac{2 \sqrt{3}}{3}\right) + d_{3} \left(\frac{7}{6} - \frac{2 \sqrt{3}}{3}\right)\right) K{\left(- \frac{\sqrt{3}}{3} \right)} + \left(\frac{d_{1}}{6} - d_{2} \left(\frac{2 \sqrt{3}}{3} + \frac{4}{3}\right) + d_{3} \left(\frac{2 \sqrt{3}}{3} + \frac{7}{6}\right)\right) K{\left(\frac{\sqrt{3}}{3} \right)}$

In [8]:
for a in range(3) :
    display( Markdown( f"$f_{a+1}^e = {sympy.latex(rat(fe[a]))}$" ))

$f_1^e = \frac{f_{1}}{6} + \frac{h}{3}$

$f_2^e = \frac{2 f_{2}}{3}$

$f_3^e = \frac{f_{3}}{6}$

In [9]:
for a in range(3) :
    for b in range(3) :      
        display( Markdown( r"$\frac{\partial n_"+f"{a+1}"+r"^e}{\partial d_"+f"{b+1}"+r"^e} $ ="+ f"${sympy.latex(rat(dna_ddb[a,b]))}$" ))
    print("\n\n")

$\frac{\partial n_1^e}{\partial d_1^e} $ =$\left(d_{1} \left(\frac{19}{36} - \frac{11 \sqrt{3}}{36}\right) - d_{2} \left(\frac{5}{9} - \frac{\sqrt{3}}{3}\right) + \frac{d_{3} \left(1 - \sqrt{3}\right)}{36}\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} + \left(d_{1} \left(\frac{19}{36} + \frac{11 \sqrt{3}}{36}\right) - d_{2} \left(\frac{5}{9} + \frac{\sqrt{3}}{3}\right) + \frac{d_{3} \left(1 + \sqrt{3}\right)}{36}\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} + \left(\frac{2 \sqrt{3}}{3} + \frac{7}{6}\right) K{\left(- \frac{\sqrt{3}}{3} \right)} + \left(\frac{7}{6} - \frac{2 \sqrt{3}}{3}\right) K{\left(\frac{\sqrt{3}}{3} \right)}$

$\frac{\partial n_1^e}{\partial d_2^e} $ =$\left(d_{1} \left(\frac{7}{9} - \frac{4 \sqrt{3}}{9}\right) - d_{2} \left(\frac{8}{9} - \frac{4 \sqrt{3}}{9}\right) + \frac{d_{3}}{9}\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} + \left(d_{1} \left(\frac{4 \sqrt{3}}{9} + \frac{7}{9}\right) - d_{2} \left(\frac{4 \sqrt{3}}{9} + \frac{8}{9}\right) + \frac{d_{3}}{9}\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} - \left(\frac{2 \sqrt{3}}{3} + \frac{4}{3}\right) K{\left(- \frac{\sqrt{3}}{3} \right)} - \left(\frac{4}{3} - \frac{2 \sqrt{3}}{3}\right) K{\left(\frac{\sqrt{3}}{3} \right)}$

$\frac{\partial n_1^e}{\partial d_3^e} $ =$\left(- d_{1} \left(\frac{5}{36} - \frac{\sqrt{3}}{12}\right) + d_{2} \left(\frac{1}{9} - \frac{\sqrt{3}}{9}\right) + \frac{d_{3} \left(1 + \sqrt{3}\right)}{36}\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} + \left(- d_{1} \left(\frac{5}{36} + \frac{\sqrt{3}}{12}\right) + d_{2} \left(\frac{1}{9} + \frac{\sqrt{3}}{9}\right) + \frac{d_{3} \left(1 - \sqrt{3}\right)}{36}\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} + \frac{K{\left(- \frac{\sqrt{3}}{3} \right)}}{6} + \frac{K{\left(\frac{\sqrt{3}}{3} \right)}}{6}$

$\frac{\partial n_2^e}{\partial d_1^e} $ =$\left(- d_{1} \left(\frac{5}{9} - \frac{\sqrt{3}}{3}\right) + \frac{4 d_{2} \left(1 - \sqrt{3}\right)}{9} + d_{3} \left(\frac{1}{9} + \frac{\sqrt{3}}{9}\right)\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} + \left(- d_{1} \left(\frac{5}{9} + \frac{\sqrt{3}}{3}\right) + \frac{4 d_{2} \left(1 + \sqrt{3}\right)}{9} + d_{3} \left(\frac{1}{9} - \frac{\sqrt{3}}{9}\right)\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} - \left(\frac{2 \sqrt{3}}{3} + \frac{4}{3}\right) K{\left(- \frac{\sqrt{3}}{3} \right)} - \left(\frac{4}{3} - \frac{2 \sqrt{3}}{3}\right) K{\left(\frac{\sqrt{3}}{3} \right)}$

$\frac{\partial n_2^e}{\partial d_2^e} $ =$- \left(d_{1} \left(\frac{8}{9} - \frac{4 \sqrt{3}}{9}\right) - \frac{16 d_{2}}{9} + d_{3} \left(\frac{4 \sqrt{3}}{9} + \frac{8}{9}\right)\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} - \left(d_{1} \left(\frac{4 \sqrt{3}}{9} + \frac{8}{9}\right) - \frac{16 d_{2}}{9} + d_{3} \left(\frac{8}{9} - \frac{4 \sqrt{3}}{9}\right)\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} + \frac{8 K{\left(- \frac{\sqrt{3}}{3} \right)}}{3} + \frac{8 K{\left(\frac{\sqrt{3}}{3} \right)}}{3}$

$\frac{\partial n_2^e}{\partial d_3^e} $ =$\left(d_{1} \left(\frac{1}{9} - \frac{\sqrt{3}}{9}\right) + \frac{4 d_{2} \left(1 + \sqrt{3}\right)}{9} - d_{3} \left(\frac{5}{9} + \frac{\sqrt{3}}{3}\right)\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} + \left(d_{1} \left(\frac{1}{9} + \frac{\sqrt{3}}{9}\right) + \frac{4 d_{2} \left(1 - \sqrt{3}\right)}{9} - d_{3} \left(\frac{5}{9} - \frac{\sqrt{3}}{3}\right)\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} - \left(\frac{4}{3} - \frac{2 \sqrt{3}}{3}\right) K{\left(- \frac{\sqrt{3}}{3} \right)} - \left(\frac{2 \sqrt{3}}{3} + \frac{4}{3}\right) K{\left(\frac{\sqrt{3}}{3} \right)}$

$\frac{\partial n_3^e}{\partial d_1^e} $ =$\left(\frac{d_{1} \left(1 - \sqrt{3}\right)}{36} + d_{2} \left(\frac{1}{9} + \frac{\sqrt{3}}{9}\right) - d_{3} \left(\frac{5}{36} + \frac{\sqrt{3}}{12}\right)\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} + \left(\frac{d_{1} \left(1 + \sqrt{3}\right)}{36} + d_{2} \left(\frac{1}{9} - \frac{\sqrt{3}}{9}\right) - d_{3} \left(\frac{5}{36} - \frac{\sqrt{3}}{12}\right)\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} + \frac{K{\left(- \frac{\sqrt{3}}{3} \right)}}{6} + \frac{K{\left(\frac{\sqrt{3}}{3} \right)}}{6}$

$\frac{\partial n_3^e}{\partial d_2^e} $ =$\left(\frac{d_{1}}{9} - d_{2} \left(\frac{8}{9} - \frac{4 \sqrt{3}}{9}\right) + d_{3} \left(\frac{7}{9} - \frac{4 \sqrt{3}}{9}\right)\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} + \left(\frac{d_{1}}{9} - d_{2} \left(\frac{4 \sqrt{3}}{9} + \frac{8}{9}\right) + d_{3} \left(\frac{4 \sqrt{3}}{9} + \frac{7}{9}\right)\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} - \left(\frac{4}{3} - \frac{2 \sqrt{3}}{3}\right) K{\left(- \frac{\sqrt{3}}{3} \right)} - \left(\frac{2 \sqrt{3}}{3} + \frac{4}{3}\right) K{\left(\frac{\sqrt{3}}{3} \right)}$

$\frac{\partial n_3^e}{\partial d_3^e} $ =$\left(\frac{d_{1} \left(1 - \sqrt{3}\right)}{36} - d_{2} \left(\frac{5}{9} - \frac{\sqrt{3}}{3}\right) + d_{3} \left(\frac{19}{36} - \frac{11 \sqrt{3}}{36}\right)\right) K_{,u}{\left(- \frac{\sqrt{3}}{3} \right)} + \left(\frac{d_{1} \left(1 + \sqrt{3}\right)}{36} - d_{2} \left(\frac{5}{9} + \frac{\sqrt{3}}{3}\right) + d_{3} \left(\frac{19}{36} + \frac{11 \sqrt{3}}{36}\right)\right) K_{,u}{\left(\frac{\sqrt{3}}{3} \right)} + \left(\frac{7}{6} - \frac{2 \sqrt{3}}{3}\right) K{\left(- \frac{\sqrt{3}}{3} \right)} + \left(\frac{2 \sqrt{3}}{3} + \frac{7}{6}\right) K{\left(\frac{\sqrt{3}}{3} \right)}$